---

### THIS NOTEBOOK EXPLAIN HOW WE DESIGNED THE LINK BETWEEN THE API, THE AZURE DATABASE, AND THE PYTHON FILE

---
### Imports

In [1]:
import json
import pandas as pd
import numpy as np
import tweepy
from azure.cosmos import exceptions, CosmosClient, PartitionKey
import asyncio

---

### Connect to Azure Cosmos DB

In [2]:
# <add_uri_and_key>
with open('venv/keys.json') as f:
    keys = json.load(f)

endpoint = keys['endpoint']
key = keys['key']
# </add_uri_and_key>

# <define_database_and_container_name>
database_name = 'DataCampCOS'
container_name = 'Tweets'
# </define_database_and_container_name>

In [3]:
# Connection to the database
client = CosmosClient(endpoint, key)

---

# Adding tweets from company name input
*input* : 
* `company_name` (string) 
* `nbr_of_tweets` (int)


*output* : 
* `tweets` (list of strings)

In [4]:
company_name = 'Lego'
nbr_of_tweets = 1000

### Get tweets from API of twitter

In [5]:
# get keys from keys.json
with open('venv/keys.json') as f:
    keys = json.load(f)

# get the keys from the dictionary
consumer_key = keys['consumer_key']
consumer_secret = keys['consumer_secret']
access_token = keys['access_token']
access_token_secret = keys['access_token_secret']

> connect to API

In [6]:
# authentification to twitter with twitter api v2
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

# check if authentification is successful
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


### Get tweets from the api and store them in a json format

In [7]:
i= 0
tweets_json = []
while i < nbr_of_tweets/100:
    tweets = api.search_tweets(q=company_name + " -RT -giveaway", count = 100,tweet_mode="extended",lang="en")
    for tweet in tweets:
        tweets_json.append(tweet._json)
    i += 1 

> Our tweets are stored in a json format --> `tweets_json`

---

### Add tweets to the database

In [8]:
print("Number of tweets to be added : ",len(tweets_json))

Number of tweets to be added :  1000


In [9]:
# change the type of id of tweets to string
for tweet in tweets_json:
    tweet['id'] = str(tweet['id'])

# add a new column to the tweets_dict called 'company' and assign the value 'uber'
for tweet in tweets_json:
    tweet['company'] = company_name

---

### We create a new item named by the company name to store all tweets

In [10]:
#! Create an item to store tweets from a new company
try:
    client.get_database_client(database_name).get_container_client(container_name).create_item(body={'id': company_name, 'tweets': []})
except exceptions.CosmosResourceExistsError:
    print("Item already exists")

---

### Then we can acces it and add new tweets

In [ ]:
item = client.get_database_client(database_name).get_container_client(container_name).read_item(item=company_name, partition_key=company_name)

# add the tweets to the item
item['tweets'] = tweets_json

# update the item
try:
    client.get_database_client(database_name).get_container_client(container_name).upsert_item(body=item)
except exceptions.CosmosHttpResponseError:
    print("Error")

---
We look how many tweets are stored in the database

In [ ]:
# check how many tweets are in the list tweets of the item Uber
item = client.get_database_client(database_name).get_container_client(container_name).read_item(item=company_name, partition_key=company_name)
print("Number of tweets in the list tweets of the item", company_name, ":", len(item['tweets']))

---